# Interactive Viz

This is our implementation of an interactive vizualization showing differences between the grants amount accorded to university and institute, according to Canton, using P3 database of the SNSF for the grant data.

We use Folium to display the choropleth maps, Google places and GeoName API to link universities with cantons.

In [10]:
%matplotlib inline
import pandas as pd
import numpy as np
import scipy.stats as stats
import folium
import os
import json
import geopy
import geocoder
import time
import csv
from IPython.display import IFrame
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
#pip install python-google-places


First import the data

In [4]:
dataCrow = pd.read_csv("data/players_cleaned_with_feature.csv", sep=',')

In [5]:
dataCrow.ix[:8,:14]

,playerShort,club,leagueCountry,birthday,position,redCards,meanIAT_redCards,ties,meanExp_yellowReds,yellowCards,meanIAT_yellowCards,meanExp_redCards,skin_color,height
0,aaron-hughes,Fulham FC,England,3598.0,Center Back,0,-0.001420,179,-0.001615,19,-0.034618,-0.020220,0.125,182.0
1,aaron-hunt,Werder Bremen,Germany,6090.0,Attacking Midfielder,1,-0.000453,73,-0.001535,42,-0.002753,0.000029,0.125,183.0
2,aaron-lennon,Tottenham Hotspur,England,6314.0,Right Midfielder,0,-0.001416,97,-0.001610,11,-0.035463,-0.020464,0.250,165.0
3,aaron-ramsey,Arsenal FC,England,7664.0,Center Midfielder,1,-0.000147,42,-0.001747,31,-0.004654,-0.008712,0.000,178.0
4,abdelhamid-el-kaoutari,Montpellier HSC,France,7380.0,Center Back,2,0.003984,40,0.010400,8,-0.022667,0.020312,0.250,180.0
5,abdou-traore_2,Girondins Bordeaux,France,6590.0,Right Midfielder,0,-0.001398,23,0.002186,11,-0.005505,-0.004792,0.750,180.0
6,abdoulaye-diallo_2,Stade Rennes,France,8124.0,Goalkeeper,0,-0.001464,8,-0.001726,0,-0.045816,-0.002628,0.875,189.0
7,abdoulaye-keita_2,Girondins Bordeaux,France,7535.0,Goalkeeper,0,-0.001485,1,-0.001573,0,-0.046475,-0.002618,0.875,188.0
8,abdoulwhaid-sissoko,Stade Brest,France,7383.0,Defensive Midfielder,2,0.004065,25,-0.001622,21,0.014230,0.031430,1.000,180.0


In [6]:
dataCrow.ix[:8,15:]

,victories,meanIAT_yellowReds,defeats,games,goals,weight
0,247,-0.001343,228,654,9,71.0
1,141,-0.001376,122,336,62,73.0
2,200,-0.001340,115,412,31,63.0
3,150,-0.001357,68,260,39,76.0
4,41,0.009606,43,124,1,73.0
5,41,0.002128,33,97,3,74.0
6,8,-0.001385,8,24,0,80.0
7,0,-0.001405,2,3,0,83.0
8,34,-0.001387,62,121,3,68.0


In [11]:
#X = grouped
X=dataCrow
for i in range(2,15):
    kmeans_model = KMeans(n_clusters=2, random_state=1).fit(X[X.columns[:i]])
    labels = kmeans_model.labels_
    print(np.mean(labels))

ValueError: could not convert string to float: 'Real Sociedad'